In [182]:

import pkg_resources
import subprocess
import openai
import re
import json

def install_if_needed(package):
    try:
        dist = pkg_resources.get_distribution(package)
        print(f"{package} is already installed (version {dist.version}).")
    except pkg_resources.DistributionNotFound:
        print(f"{package} is not installed. Installing...")
        subprocess.call(['pip3', 'install', package])
        print(f"{package} has been installed.")
        
install_if_needed('openai')

openai is already installed (version 0.27.8).


### USEFUL LINKS

- https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
- https://www.youtube.com/watch?v=uCKH8bmPgFs&t=502s
- https://github.com/openai/openai-cookbook

In [184]:
openai.api_type = "azure"
openai.api_version = "2023-05-15"

#HIDE
openai.api_base = "https://gpt4resource.openai.azure.com/" # Your Azure OpenAI resource's endpoint value.
openai.api_key = "618bf90290544e54b47bcfc2dba743da"

In [265]:
language = 'Node.js'
topic = 'web development'
difficulty = 'intermediate'

In [266]:
system_message = 'Assistant is an intelligent chatbot designed to help users create programming training materials'
user_message = f"""Could you please provide me with incorrect code in {language}, followed by a corresponding unit test that will fail? 
                        Then provide corrected code that will pass the unit test. 
                        For each code snippet, provide an appropriate file name. 
                        The area of interest is {topic}. 
                        The difficulty should be {difficulty}."""

In [267]:
print(user_message)

Could you please provide me with incorrect code in Node.js, followed by a corresponding unit test that will fail? 
                        Then provide corrected code that will pass the unit test. 
                        For each code snippet, provide an appropriate file name. 
                        The area of interest is web development. 
                        The difficulty should be intermediate.


In [268]:
response = openai.ChatCompletion.create(
    #gpt-35-turbo max tokens = 4,096
    engine="testdeploy", # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ],
    temperature = 0 # This is the degree of randomness of the model's output
)

In [269]:
# print(response)
# print(50*'-')

content = response['choices'][0]['message']['content']

print('tokens: ', response['usage'])
print(100*'-')
print(100*'-')
print(content)

tokens:  {
  "completion_tokens": 264,
  "prompt_tokens": 93,
  "total_tokens": 357
}
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Sure, here's an example:

**Incorrect Code:**

File Name: `server.js`

```javascript
const express = require('express');
const app = express();

app.get('/', (req, res) => {
  res.send('Hello World');
});

app.listen(3000);
```

**Unit Test:**

File Name: `test-server.js`

```javascript
const request = require('supertest');
const app = require('./server');

describe('GET /', () => {
  it('responds with "Hello World"', (done) => {
    request(app)
      .get('/')
      .expect('Hello World')
      .end(done);
  });
});
```

**Corrected Code:**

File Name: `server.js`

```javascript
const express = require('express');
const app = express();

app.get('/', (req, res) => {
  res.send('Hello World');
});

con

In [273]:
def extract_languages(text):
    pattern = r"```(.*)\n"
    matches = re.findall(pattern, text)
    languages = [match for match in matches if match != '']
    return languages

def extract_filenames(text):
    pattern = r": `(.*)`\n"
    matches = re.findall(pattern, text)
    matches.pop()
    matches.append('corrected-' + matches[0].split('.')[0] + '.' + matches[0].split('.')[1])
    return matches

def remove_before_second_newline(text):
    lines = text.split('\n')
    new_text = '\n'.join(lines[2:])
    if new_text[0] == '\n':
        return new_text[1:]
    else:
        return new_text

def extract_code(text):
    pattern = r"```([\s\S]*?)```"
    matches = re.findall(pattern, text)
    code = []
    for match in matches:
        code.append(remove_before_second_newline(match))
    return code

In [274]:
languages = extract_languages(content)
filenames = extract_filenames(content)
code = extract_code(content)

In [275]:
languages

['javascript', 'javascript', 'javascript']

In [276]:
filenames

['server.js', 'test-server.js', 'corrected-server.js']

In [277]:
code

["const app = express();\n\napp.get('/', (req, res) => {\n  res.send('Hello World');\n});\n\napp.listen(3000);\n",
 'const app = require(\'./server\');\n\ndescribe(\'GET /\', () => {\n  it(\'responds with "Hello World"\', (done) => {\n    request(app)\n      .get(\'/\')\n      .expect(\'Hello World\')\n      .end(done);\n  });\n});\n',
 "const app = express();\n\napp.get('/', (req, res) => {\n  res.send('Hello World');\n});\n\nconst server = app.listen(3000, () => {\n  console.log(`Server running on port ${server.address().port}`);\n});\n\nmodule.exports = app;\n"]

In [278]:
data = {
    'labels': ['Incorrect code', 'Unit test', 'Corrected code'],
    'languages': languages,
    'filenames': filenames,
    'code': code
}

json_filename = 'label_lang_file_code.json'
with open(json_filename, 'w') as file:
    json.dump(data, file)

In [28]:
#RUN CELL TO SAVE CONTENT IN A TXT FILE
import os

folder_path = 'txt_outputs'
text_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]
txt_number_list = []
for file_name in text_files:
    txt_number = file_name[-5]
    txt_number = int(txt_number)
    txt_number_list.append(txt_number)
if not txt_number_list: #check if list is empty, i.e. no txt files yet
    new_txt_number = str(1)
else:
    new_txt_number = str(max(txt_number_list) + 1)
text_file = open('txt_outputs/output' + '_' + new_txt_number + '.txt', 'w')
text_file.write(system_message + '\n')
text_file.write(50*'-' + '\n')
text_file.write(user_message + '\n')
text_file.write(50*'-' + '\n')
text_file.write(content + '\n')
text_file.close()